In [1]:
!nvidia-smi


Sun Feb  8 10:24:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
!pip install -q \
  torch \
  transformers \
  peft \
  accelerate \
  fastapi \
  uvicorn \
  nest-asyncio \
  pyngrok


In [4]:
!unzip phi3_medical_lora_bin.zip


Archive:  phi3_medical_lora_bin.zip
   creating: phi3_medical_lora_bin/
  inflating: phi3_medical_lora_bin/adapter_config.json  
  inflating: phi3_medical_lora_bin/README.md  
  inflating: phi3_medical_lora_bin/adapter_model.bin  


In [5]:
!ls phi3_medical_lora_bin


adapter_config.json  adapter_model.bin	README.md


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "microsoft/phi-3-mini-4k-instruct"
LORA_PATH = "phi3_medical_lora_bin"

print("🔹 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

print("🔹 Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("🔹 Loading LoRA adapter...")
model = PeftModel.from_pretrained(
    model,
    LORA_PATH,
    is_trainable=False
)

model.eval()
print("✅ Fine-tuned model loaded successfully")


🔹 Loading tokenizer...
🔹 Loading base model...


Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

🔹 Loading LoRA adapter...
✅ Fine-tuned model loaded successfully


In [24]:
prompt = """### Instruction:
Answer in a professional medical tone.

Question:
What are first-line treatments for hypertension?

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Answer in a professional medical tone.

Question:
What are first-line treatments for hypertension?

### Response:
First-line treatment decisions depend on patient-specific factors.


In [25]:
nest_asyncio.apply()

app = FastAPI(title="Fine-tuned Phi-3 API")

class ChatRequest(BaseModel):
    prompt: str

@app.post("/chat")
def chat(req: ChatRequest):
    inputs = tokenizer(req.prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            temperature=0.7
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 🔥 TAKE ONLY WHAT COMES AFTER THE LAST "Answer:"
    if "Answer:" in decoded:
        decoded = decoded.rsplit("Answer:", 1)[-1]

    decoded = decoded.strip()

    return {"response": decoded}








In [32]:
!pkill ngrok


In [33]:

from pyngrok import ngrok
ngrok.set_auth_token("39NqGuTOmRcLcHBm0317XJcY4XF_2js7tnkRnJqFYeowWMDDJ")



In [35]:

from pyngrok import ngrok
import threading

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run)
thread.start()


🌍 Public URL: NgrokTunnel: "https://prolongedly-unrevelling-lorri.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [277]
INFO:     Waiting for application startup.
